## Modelos seq2seq + atención


Los modelos de atención han transformado la forma en que abordamos las tareas de traducción automática y otras aplicaciones de procesamiento de lenguaje natural (NLP). Estos modelos permiten que el decodificador se centre en diferentes partes de la secuencia de entrada mientras genera cada palabra de la secuencia de salida.



Dos mecanismos importantes en este contexto son la atención global y la atención local. Este cuaderno detalla estos mecanismos, sus implementaciones, y sus aplicaciones prácticas.

#### Mecanismo de atención global

<img src="https://lilianweng.github.io/posts/2018-06-24-attention/encoder-decoder-attention.png" width="600">


**Descripción general**

El mecanismo de atención global, introducido por Bahdanau también conocido como atención suave, permite que el decodificador considere todas las posiciones de la secuencia de entrada al generar cada palabra de la secuencia de salida. Este enfoque asegura que el modelo tenga acceso a toda la información de la entrada en cada paso del proceso de decodificación, mejorando la calidad de la traducción, especialmente en secuencias largas y complejas.

**Ecuaciones y cálculos**

El mecanismo de atención global se basa en los siguientes pasos y ecuaciones:

1. **Cálculo de los puntajes de atención**:
   Para cada paso de tiempo del decodificador, se calcula un puntaje de atención $ e_{ij}$ que mide la afinidad entre el estado oculto del decodificador en el paso de tiempo $ j$, denotado como $ s_{j-1}$, y el estado oculto del codificador en el paso de tiempo $ i$, denotado como $ h_i$. Esto se puede calcular usando una red neuronal feedforward con una sola capa oculta (o cualquier otra función de afinidad):

   $$
   e_{ij} = v^T \tanh(W_1 h_i + W_2 s_{j-1})
   $$

   donde $W_1$ y $W_2$ son matrices de peso aprendibles y $v$ es un vector de peso aprendible.

2. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $ \alpha_{ij}$, que son distribuciones de probabilidad sobre las posiciones de la secuencia de entrada:

 $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}
  $$

3. **Cálculo del vector de contexto**:
   El vector de contexto $ c_j$ para cada paso de tiempo del decodificador se calcula como una combinación ponderada de los estados ocultos del codificador:

 $$
   c_j = \sum_{i=1}^{T_x} \alpha_{ij} h_i
 $$

4. **Generación de la salida del decodificador**:
   Finalmente, el vector de contexto $ c_j$ se combina con el estado oculto del decodificador $ s_j$ para generar la salida $ y_j$:

 $$
   y_j = g(c_j, s_j)
 $$

  donde $ g$ puede ser una función no lineal como una red neuronal.
    


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención global dentro de un modelo seq2seq:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        # Capa lineal para calcular la energía de la atención
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        # Vector de parámetros de atención
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(1)  # Longitud de la secuencia de entrada
        H = hidden.repeat(max_len, 1, 1).transpose(0, 1)  # Expande el estado oculto del decodificador
        
        # Calcula la energía de atención
        energy = torch.tanh(self.attn(torch.cat([H, encoder_outputs], 2)))
        energy = energy.transpose(1, 2)  # Ajusta la dimensión para la multiplicación
        
        # Calcula los pesos de atención
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # Expande el vector de atención
        energy = torch.bmm(v, energy)  # Multiplicación de matrices batch
        attn_weights = F.softmax(energy.squeeze(1), dim=1)  # Aplica softmax para normalizar
        return attn_weights

class Seq2SeqWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithAttention, self).__init__()
        
        # Definición del codificador con GRU
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        
        # Definición del decodificador con GRU
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        
        # Módulo de atención
        self.attention = Attention(hidden_dim)
        
        # Capa lineal para generar la salida
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, src, trg):
        # Codificación de la secuencia de entrada
        encoder_outputs, hidden = self.encoder(src)
        
        # Decodificación de la secuencia de salida
        decoder_outputs, hidden = self.decoder(trg, hidden)
        
        # Aplicación de la atención
        attn_weights = self.attention(hidden, encoder_outputs)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)  # Multiplicación de los pesos de atención con las salidas del codificador
        
        # Concatenación del contexto y el estado oculto
        output = torch.cat([hidden.squeeze(0), context.squeeze(1)], 1)
        output = self.fc(output)  # Capa final para generar la salida
        return output


#### Mecanismo de atención local

**Descripción general**

El mecanismo de atención local, propuesto por Luong (2015), reduce la complejidad computacional al limitar el alcance de la atención a una ventana local alrededor de cada posición de la secuencia de entrada. Este enfoque es particularmente útil en secuencias largas, donde la atención global puede ser computacionalmente costosa.

**Ecuaciones y cálculos**

El mecanismo de atención local se define a través de los siguientes pasos:

1. **Predicción de la posición de atención**:
   Primero, se predice una posición de atención $p_j$ para cada paso de tiempo $j$ del decodificador. Esto puede hacerse mediante una simple función lineal o una red neuronal:
   

   $$
   p_j = S \cdot \sigma(W_p s_{j-1})
   $$ 

   donde $S$ es la longitud de la secuencia de entrada, $\sigma$ es la función sigmoide, $W_p$ es una matriz de peso aprendible, y $s_{j-1}$ es el estado oculto del decodificador en el paso $j-1$.


2. **Definición de la ventana local**:
   Se define una ventana local de tamaño $2D + 1$ centrada en $p_j$. Los límites de la ventana se calculan como:

   $$
   [p_j - D, p_j + D]
  $$ 


3. **Cálculo de puntajes de atención dentro de la ventana**:
   Los puntajes de atención $e_{ij}$ se calculan solo para las posiciones dentro de la ventana local:


   $$
   e_{ij} = v^T \tanh(W_1 h_i + W_2 s_{j-1})
  $$ 


4. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $\alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k \in [p_j - D, p_j + D]} \exp(e_{ik})}
  $$ 

5. **Cálculo del vector de contexto**:
   El vector de contexto $c_j$ se calcula como una combinación ponderada de los estados ocultos del codificador dentro de la ventana local:

   $$
   c_j = \sum_{i \in [p_j - D, p_j + D]} \alpha_{ij} h_i
  $$ 

<img src="https://lilianweng.github.io/posts/2018-06-24-attention/luong2015-fig2-3.png" width="600">


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención local dentro de un modelo seq2seq:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LocalAttention(nn.Module):
    def __init__(self, hidden_size, window_size):
        super(LocalAttention, self).__init__()
        self.window_size = window_size  # Tamaño de la ventana de atención local
        self.attn = nn.Linear(hidden_size * 2, hidden_size)  # Capa lineal para calcular energía de atención
        self.v = nn.Parameter(torch.rand(hidden_size))  # Vector de parámetros de atención

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)  # Tamaño del batch
        max_len = encoder_outputs.size(1)  # Longitud máxima de la secuencia de entrada
        hidden = hidden.squeeze(0)  # Ajusta la dimensión del estado oculto
        
        # Posiciones de las palabras en la secuencia
        pos = torch.arange(max_len).unsqueeze(0).repeat(batch_size, 1)
        
        # Expande el estado oculto del decodificador
        hidden = hidden.unsqueeze(1).repeat(1, max_len, 1)
        
        # Calcula la energía de la atención
        attn_energies = torch.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
        attn_energies = torch.sum(self.v * attn_energies, dim=2)  # Ponderación con el vector de atención
        
        # Calcula los pesos de atención aplicando softmax
        attn_weights = F.softmax(attn_energies, dim=1)
        
        return attn_weights

class Seq2SeqWithLocalAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, window_size):
        super(Seq2SeqWithLocalAttention, self).__init__()
        
        # Definición del codificador con GRU
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        
        # Definición del decodificador con GRU
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        
        # Módulo de atención local
        self.local_attention = LocalAttention(hidden_dim, window_size)
        
        # Capa lineal para generar la salida
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, src, trg):
        # Codificación de la secuencia de entrada
        encoder_outputs, hidden = self.encoder(src)
        
        # Decodificación de la secuencia de salida
        decoder_outputs, hidden = self.decoder(trg, hidden)
        
        # Aplicación de la atención local
        attn_weights = self.local_attention(hidden, encoder_outputs)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)  # Multiplicación de los pesos de atención con las salidas del codificador
        
        # Concatenación del contexto y el estado oculto
        output = torch.cat([hidden.squeeze(0), context.squeeze(1)], 1)
        output = self.fc(output)  # Capa final para generar la salida
        return output


#### Mecanismo de atención jerárquica

**Descripción general**

La atención jerárquica se utiliza para manejar estructuras de datos complejas y de múltiples niveles, como documentos largos divididos en párrafos, párrafos divididos en oraciones y oraciones divididas en palabras. Este mecanismo aplica la atención en dos niveles: a nivel de palabra dentro de cada oración y a nivel de oración dentro del documento. Esta estructura permite capturar dependencias tanto locales como globales de manera eficiente.

**Ecuaciones y cálculos**

El proceso de atención jerárquica se puede dividir en dos fases principales:

1. **Atención a nivel de palabra**:
   Primero, se aplica la atención para cada palabra dentro de cada oración. Supongamos que una oración $ \text{sentence}_i $ contiene $ T_i $ palabras y sus representaciones de palabra son $ \{h_{i1}, h_{i2}, \ldots, h_{iT_i}\} $. La atención a nivel de palabra se calcula de la siguiente manera:

   $$
   e_{ij} = v_1^T \tanh(W_1 h_{ij} + b_1)
   $$

   donde $ W_1 $ y $ v_1 $ son parámetros aprendibles, y $ b_1 $ es un vector de sesgo.

   Los pesos de atención se obtienen aplicando la función softmax a los puntajes $ e_{ij} $:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_i} \exp(e_{ik})}
   $$

   El vector de contexto para la oración $ i $ se calcula como una combinación ponderada de las representaciones de palabra:

   $$
   c_i = \sum_{j=1}^{T_i} \alpha_{ij} h_{ij}
   $$

2. **Atención a nivel de oración**:
   Una vez obtenidos los vectores de contexto $ \{c_1, c_2, \ldots, c_N\} $ para todas las oraciones en un documento (donde $ N $ es el número de oraciones en el documento), se aplica la atención a nivel de oración:

   $$
   e_i = v_2^T \tanh(W_2 c_i + b_2)
   $$

   Los pesos de atención a nivel de oración se obtienen aplicando la función softmax a los puntajes $ e_i $:

   $$
   \beta_i = \frac{\exp(e_i)}{\sum_{k=1}^{N} \exp(e_k)}
   $$

   El vector de contexto para el documento se calcula como una combinación ponderada de los vectores de contexto de las oraciones:

   $$
   d = \sum_{i=1}^{N} \beta_i c_i
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención jerárquica:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HierarchicalAttention(nn.Module):
    def __init__(self, word_hidden_size, sentence_hidden_size, vocab_size, word_embedding_dim, num_classes):
        super(HierarchicalAttention, self).__init__()
        
        # Capa de embeddings para representar palabras en un espacio de menor dimensión
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        
        # Codificador a nivel de palabra con una GRU
        self.word_encoder = nn.GRU(word_embedding_dim, word_hidden_size, batch_first=True)
        
        # Módulo de atención para dar peso a cada palabra
        self.word_attention = nn.Linear(word_hidden_size, word_hidden_size)
        
        # Codificador a nivel de oración con una GRU
        self.sentence_encoder = nn.GRU(word_hidden_size, sentence_hidden_size, batch_first=True)
        
        # Módulo de atención para dar peso a cada oración
        self.sentence_attention = nn.Linear(sentence_hidden_size, sentence_hidden_size)
        
        # Capa lineal final para clasificación u otras tareas
        self.fc = nn.Linear(sentence_hidden_size, num_classes)  # num_classes depende de la tarea específica

    def forward(self, documents):
        sentence_vectors = []  # Lista para almacenar los vectores de cada oración
        
        for sentences in documents:  # Iterar sobre cada documento
            word_vectors = []  # Lista para almacenar los vectores de palabras dentro de una oración
            
            for sentence in sentences:  # Iterar sobre cada oración en el documento
                
                # Obtener las representaciones de las palabras mediante embeddings
                embedded_words = self.word_embedding(sentence)
                
                # Aplicar la GRU a nivel de palabra
                word_enc_outputs, _ = self.word_encoder(embedded_words)
                
                # Calcular pesos de atención a nivel de palabra
                word_att_weights = F.softmax(self.word_attention(word_enc_outputs), dim=1)
                
                # Obtener el vector de la oración ponderando las palabras
                word_vector = torch.sum(word_att_weights * word_enc_outputs, dim=1)
                word_vectors.append(word_vector)
            
            # Almacenar los vectores de oraciones en la lista de vectores de oraciones
            sentence_vectors.append(torch.stack(word_vectors))
        
        # Aplicar la GRU a nivel de oración
        sentence_enc_outputs, _ = self.sentence_encoder(torch.stack(sentence_vectors))
        
        # Calcular pesos de atención a nivel de oración
        sentence_att_weights = F.softmax(self.sentence_attention(sentence_enc_outputs), dim=1)
        
        # Obtener el vector del documento ponderando las oraciones
        doc_vector = torch.sum(sentence_att_weights * sentence_enc_outputs, dim=1)
        
        # Aplicar la capa final para obtener la salida
        output = self.fc(doc_vector)
        
        return output


#### Mecanismo de atención basada en consultas

**Descripción general**

La atención basada en consultas, utilizada en modelos como el Transformer, utiliza tres componentes principales: consultas (queries), claves (keys) y valores (values). Este mecanismo permite calcular la atención como una función de similitud entre las consultas y las claves, aplicándola a los valores para obtener una representación ponderada. Este enfoque es altamente eficiente y escalable.

**Ecuaciones y cálculos**

El mecanismo de atención basada en consultas se define a través de los siguientes pasos:

1. **Cálculo de consultas, claves y valores**:
   Las consultas $ Q$, las claves $ K$ y los valores $ V$ se obtienen mediante proyecciones lineales de la entrada:

   $$
   Q = X W_Q, \quad K = X W_K, \quad V = X W_V
   $$

   donde $ W_Q$, $ W_K$ y $ W_V$ son matrices de peso aprendibles.

2. **Cálculo de puntajes de atención**:
   Los puntajes de atención se calculan como el producto punto escalado entre las consultas y las claves:

   $$
   e_{ij} = \frac{Q_i K_j^T}{\sqrt{d_k}}
   $$

   donde $ d_k$ es la dimensión de las claves.

3. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $ \alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k} \exp(e_{ik})}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto $ c_i$ se calcula como una combinación ponderada de los valores:

   $$
   c_i = \sum_{j} \alpha_{ij} V_j
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención basada en consultas

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class QueryBasedAttention(nn.Module):
    def __init__(self, hidden_size):
        super(QueryBasedAttention, self).__init__()
        
        # Proyecciones lineales para query, key y value
        self.linear_q = nn.Linear(hidden_size, hidden_size)  # Proyección de la consulta (query)
        self.linear_k = nn.Linear(hidden_size, hidden_size)  # Proyección de la clave (key)
        self.linear_v = nn.Linear(hidden_size, hidden_size)  # Proyección del valor (value)
        
        # Capa lineal final para procesar la salida de la atención
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value):
        # Proyectar query, key y value en espacios latentes
        Q = self.linear_q(query)
        K = self.linear_k(key)
        V = self.linear_v(value)
        
        # Calcular los puntajes de atención mediante el producto punto escalado
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        
        # Aplicar softmax para obtener los pesos de atención
        attn_weights = F.softmax(attn_scores, dim=-1)
        
        # Multiplicar los pesos de atención con los valores (V)
        attn_output = torch.matmul(attn_weights, V)
        
        # Aplicar la capa final para generar la salida
        output = self.fc_out(attn_output)
        
        return output

class Seq2SeqWithQueryBasedAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithQueryBasedAttention, self).__init__()
        
        # Definición del codificador con GRU
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        
        # Definición del decodificador con GRU
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        
        # Módulo de atención basado en query
        self.query_attn = QueryBasedAttention(hidden_dim)
        
        # Capa lineal final para generar la salida
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, trg):
        # Codificación de la secuencia de entrada
        enc_output, hidden = self.encoder(src)
        
        # Decodificación de la secuencia de salida
        dec_output, _ = self.decoder(trg, hidden)
        
        # Aplicación de la atención basada en query
        attn_output = self.query_attn(dec_output, enc_output, enc_output)
        
        # Generar la salida final con la capa lineal
        output = self.fc(attn_output)
        
        return output


#### Mecanismo de auto-atención (Self-Attention)

**Descripción general**

El mecanismo de auto-atención, o self-attention, permite que cada elemento de la secuencia preste atención a todos los demás elementos de la misma secuencia. Esto es fundamental para capturar las dependencias a largo plazo en las secuencias y es un componente clave en los modelos Transformer.

**Ecuaciones y cálculos**

El proceso de auto-atención se puede describir mediante los siguientes pasos:

1. **Proyección lineal**:
   Al igual que en la atención multi-cabecera, se proyectan las consultas $Q$, las claves $K$ y los valores $V$:

   $$
   Q = X W_Q, \quad K = X W_K, \quad V = X W_V
   $$

   donde $W_Q$, $W_K$ y $W_V$ son matrices de peso aprendibles.

2. **Cálculo de puntajes de atención**:
   Los puntajes de atención se calculan utilizando el producto punto escalado:

   $$
   e_{ij} = \frac{Q_i K_j^T}{\sqrt{d_k}}
   $$

3. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención  $\alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k} \exp(e_{ik})}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto $c_i$ se calcula como una combinación ponderada de los valores:

   $$
   c_i = \sum_{j} \alpha_{ij} V_j
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de auto-atención:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class SelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super(SelfAttention, self).__init__()
        
        # Proyecciones lineales para query (Q), key (K) y value (V)
        self.linear_q = nn.Linear(hidden_size, hidden_size)  # Proyección de la consulta (query)
        self.linear_k = nn.Linear(hidden_size, hidden_size)  # Proyección de la clave (key)
        self.linear_v = nn.Linear(hidden_size, hidden_size)  # Proyección del valor (value)
        
        # Capa lineal final para procesar la salida de la atención
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        # Proyectar la entrada en query (Q), key (K) y value (V)
        Q = self.linear_q(x)
        K = self.linear_k(x)
        V = self.linear_v(x)

        # Calcular los puntajes de atención usando el producto punto escalado
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        
        # Aplicar softmax para obtener los pesos de atención
        attn_weights = F.softmax(attn_scores, dim=-1)
        
        # Multiplicar los pesos de atención con los valores (V)
        attn_output = torch.matmul(attn_weights, V)
        
        # Aplicar la capa lineal final para generar la salida
        output = self.fc_out(attn_output)
        
        return output


#### Mecanismo de atención multi-cabecera (Multi-Head Attention)


<img src="https://lilianweng.github.io/posts/2018-06-24-attention/transformer.png" width="600">

**Descripción general**

El mecanismo de atención multi-cabecera, introducido por Vaswani en el modelo Transformer, extiende la idea de la auto-atención al permitir que el modelo se concentre en diferentes partes de la secuencia de entrada de manera simultánea y desde múltiples perspectivas. Esto se logra al tener múltiples "cabeceras" de atención, cada una de las cuales realiza una operación de atención independiente.

**Ecuaciones y cálculos**

El proceso de atención multi-cabecera se puede dividir en varios pasos:

1. **Proyección lineal**:
   Se proyectan las consultas  $Q$, las claves $K$ y los valores $V$ en subespacios diferentes para cada cabecera de atención. Supongamos que tenemos $h$ cabecera de atención y una dimensión de modelo $d_{\text{model}}$. La proyección se realiza de la siguiente manera:

   $$
   Q_h = X W_Q^h, \quad K_h = X W_K^h, \quad V_h = X W_V^h
   $$

   donde $W_Q^h$, $W_K^h$ y $W_V^h$ son matrices de peso específicas para la cabecera  $h$.

2. **Cálculo de puntajes de atención**:
   Para cada cabecera de atención, se calcula el puntaje de atención utilizando el producto punto escalado:

   $$
   e_{ij}^h = \frac{Q_i^h (K_j^h)^T}{\sqrt{d_k}}
   $$

   donde $d_k$ es la dimensión de las claves.

3. **Normalización de puntajes de atención**:

   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $\alpha_{ij}^h$:

   $$
   \alpha_{ij}^h = \frac{\exp(e_{ij}^h)}{\sum_{k} \exp(e_{ik}^h)}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto para cada cabecera de atención se calcula como una combinación ponderada de los valores:

   $$
   c_i^h = \sum_{j} \alpha_{ij}^h V_j^h
   $$

5. **Concatenación y proyección final**:
   Los vectores de contexto de todas las cabeceras se concatenan y se proyectan de nuevo en el espacio original:

   $$
   \text{MultiHead}(Q, K, V) = \text{Concat}(c_i^1, c_i^2, \ldots, c_i^h) W_O
   $$

   donde  $W_O$ es la matriz de peso de proyección final.

La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención multi-cabecera:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        
        # Verifica que el tamaño oculto sea divisible entre el número de cabezas
        assert hidden_size % num_heads == 0
        
        self.num_heads = num_heads  # Número de cabezas de atención
        self.head_dim = hidden_size // num_heads  # Dimensión de cada cabeza

        # Proyecciones lineales para query (Q), key (K) y value (V)
        self.linear_q = nn.Linear(hidden_size, hidden_size)  # Proyección de la consulta
        self.linear_k = nn.Linear(hidden_size, hidden_size)  # Proyección de la clave
        self.linear_v = nn.Linear(hidden_size, hidden_size)  # Proyección del valor

        # Capa lineal final después de la atención multi-cabeza
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value):
        batch_size = query.size(0)  # Obtiene el tamaño del batch

        # Aplicar proyección lineal y dividir en múltiples cabezas
        Q = self.linear_q(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.linear_k(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.linear_v(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Calcular puntajes de atención escalados
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.head_dim)

        # Aplicar softmax para normalizar los puntajes de atención
        attn_weights = F.softmax(attn_scores, dim=-1)

        # Multiplicar los pesos de atención con los valores (V) para calcular el vector de contexto
        attn_output = torch.matmul(attn_weights, V)
        
        # Reorganizar las dimensiones y concatenar las cabezas de atención
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.head_dim)

        # Aplicar la proyección final para combinar la información de todas las cabezas
        output = self.fc_out(attn_output)
        
        return output


Estos mecanismos son fundamentales para el funcionamiento de los modelos Transformer, permitiendo capturar dependencias a largo plazo y manejar grandes cantidades de datos de manera eficiente. 

### Ejercicios

1 .Implementa un modelo de atención jerárquica donde primero se aplique atención a nivel de palabra y luego a nivel de frase para tareas de resumen de texto o clasificación de documentos.

Instrucciones:

- Divide un documento en oraciones y cada oración en palabras.
- Implementa atención a nivel de palabra para obtener una representación de cada oración.
- Implementa atención a nivel de frase para obtener una representación del documento.
- Usa la representación del documento para realizar una tarea específica (por ejemplo, clasificación de documentos).

In [ ]:
## Tu respuesta

2 . Implementa un mecanismo de atención multi-cabecera similar al utilizado en los modelos de transformadores, para permitir que el modelo enfoque en diferentes partes de la entrada de manera simultánea.

Instrucciones:

- Implementa un bloque de atención con múltiples cabeceras.
- Integra este bloque en un modelo seq2seq.
- Evalua el rendimiento del modelo en una tarea de traducción automática.

In [ ]:
## Tu respuesta

3 . Implementa un modelo de atención local que solo considere una ventana alrededor de la posición actual en lugar de toda la secuencia de entrada, reduciendo la complejidad computacional.

Instrucciones:

* Define una ventana de atención fija.
* Implementa el cálculo de los pesos de atención solo dentro de esta ventana.
* Integra este mecanismo en un modelo seq2seq y evaluar su rendimiento

In [ ]:
## Tu respuesta

4 . Implementa un modelo de atención basada en consultas similar al mecanismo utilizado en el Transformador, donde las consultas, las claves y los valores provienen de proyecciones de la entrada.

Instrucciones:

- Implementa las proyecciones lineales para consultas, claves y valores.
- Calcula los pesos de atención usando productos escalares entre las consultas y las claves.
- Aplica estos pesos a los valores para obtener la representación de atención

In [ ]:
## Tu respuesta

5 . Implementa la auto-atención donde cada elemento de la secuencia presta atención a todos los demás elementos de la misma secuencia. Esto es útil para tareas de traducción automática y clasificación de secuencias.

Instrucciones:

- Implementa el cálculo de auto-atención utilizando proyecciones lineales para claves, consultas y valores.
- Integra el mecanismo de auto-atención en un modelo seq2seq.
- Evalua el rendimiento del modelo en una tarea de traducción automática.

In [ ]:
## Tu respuesta